# Visão Computacional - Projeto 2-1

### Bruna Kimura

In [1]:
%matplotlib inline

import cv2 as cv2
import numpy as np     
import matplotlib.pyplot as plt

import time

### 1. Detecção de alguns pontos interessantes na imagem

In [2]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    # Converte para Grayscale
    bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Parametriza a funcao do OpenCV
    params = dict( maxCorners = 100,
                   qualityLevel = 0.3,
                   minDistance = 7,
                   blockSize = 7 )

    # Funcao que retorna uma lista de pontos
    pts = cv2.goodFeaturesToTrack(bw, mask = None, **params)
    
    # Gera cores de forma aleatória
    color = np.random.randint(0,255,(100,3))

    # Insere uma marcação em cada ponto:
    for i,pt in enumerate(pts):
        a,b = pt.ravel()
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

    cv2.imshow("Video", frame)

captura.release()
cv2.destroyAllWindows()

### 2. Rastreamento dos pontos (apenas em dois frames)

In [3]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

ret, frame = captura.read()
previous = frame
previous_gray = cv2.cvtColor(previous, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

m_x = 0
m_y = 0


while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    actual = frame
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    
    # diferença entre os pontos antigos e os atuais
    dif = np.subtract(good_old, good_new)
    
    #valores correspondentes a x e y
    x = dif[:,0]
    y = dif[:,1]
    
    #quantidade de elementos em x e y
    len_x = len(x)
    len_y = len(y)
    
    soma_x = 0
    soma_y = 0
    
    #soma todos os valores em x e y
    for e in x:
        soma_x += e
        
    for e in y:
        soma_y += e
       
    #divide a soma pela quantidade de elementos, ou seja, encontra a media de x e y 
    m_x += (soma_x/len_x)
    m_y += (soma_y/len_y)
    
    #matriz de translação (utilizando a média de x e y)
    translation_matrix = np.float32([[1,0,int(m_x)], [0,1,int(m_y)]])
    
    img = cv2.add(actual, mask)
    
    #translada a imagem
    num_rows, num_cols = img.shape[:2]
    img_translation = cv2.warpAffine(img, translation_matrix, (num_cols, num_rows))
    
    cv2.imshow('Translation', img_translation)
    
    # Now update the previous frame and previous points
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv2.destroyAllWindows()


### Dense Optical Flow

Delocar a tela conforme a pessoa se mexe para centraliza-la

In [4]:
#captura de uma imagem da camera apenas para iniciar o processo
captura = cv2.VideoCapture(0)
ret, frame1 = captura.read()

#imagem "anterior" em branco e preto
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
#encontrar o centro aproximado da imagem
prvs_center = prvs[121:341, 221:411]

#variáveis que irão armazenar o fluxo óptico na direção x e y
x_acumulado = 0
y_acumulado = 0

while(1):
    #nova captura de imagem da camera e os mesmos tratamentos de imagem acima
    ret, frame2 = captura.read()
    next1 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    next1_center = next1[121:341, 221:411]
    
    #encontrar o fluxo óptico da imagem utilizando a imagem atual e anterior
    flow = cv2.calcOpticalFlowFarneback(prvs_center,next1_center, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    #calcula a média do fluxo em ambas as direções
    x = flow[:,:,0]
    y = flow[:,:,1]
    
    x_m = np.mean(x)
    y_m = np.mean(y)
    
    #adiciona nas variáveis os valores na direção opostas
    x_acumulado -= x_m
    y_acumulado -= y_m

    #matriz de translação
    translation_matrix = np.float32([[1,0,int(x_acumulado)], [0,1,int(y_acumulado)]])

    #encontrar a quatidade de linhas e colunas da imagem
    num_rows, num_cols = frame2.shape[:2]

    #Aplicar translação na imagem
    img_translation = cv2.warpAffine(frame2, translation_matrix, (num_cols, num_rows))
    
    #mostrar a imagem transladada
    cv2.imshow('Translation', img_translation)
    
    #condição para sair da janela da camera
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
    #transformar a imagem anterior na atual imagem
    prvs_center = next1_center
    
#desligar captura de camera e fechar as janelas
captura.release()
cv2.destroyAllWindows()

Tentativa de não alterar o tamanho da imagem e tirar a barra preta lateral

In [6]:
#captura de uma imagem da camera apenas para iniciar o processo
captura = cv2.VideoCapture(0)
ret, frame1 = captura.read()

#espelhar a imagem
frame1 = cv2.flip(frame1, 1)
#imagem "anterior" em branco e preto
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

#encontrar o centro aproximado da imagem
prvs_center = prvs[121:341, 221:411]

#encontrar a quatidade de linhas e colunas da imagem
num_rows, num_cols = frame1.shape[:2]

#proporção entre as linhas e colunas
prop = num_cols/num_rows

#variáveis que irão armazenar o fluxo óptico na direção x e y
x_a = 0
y_a = 0

while(1):
    #nova captura de imagem da camera e os mesmos tratamentos de imagem acima
    ret, frame2 = captura.read()
    frame2 = cv2.flip(frame2, 1)
    next1 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    next1_center = next1[121:341, 221:411]
       
    
    #encontrar o fluxo óptico da imagem utilizando a imagem atual e anterior
    flow = cv2.calcOpticalFlowFarneback(prvs_center,next1_center, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    #calcula a média do fluxo em ambas as direções
    x_m = np.mean(flow[:,:,0])
    y_m = np.mean(flow[:,:,1])
    
    #adiciona nas variáveis os valores na direção opostas
    x_m*=(-1)
    y_m*=(-1)
    
    x_a += x_m
    y_a += y_m
    
    x_acumulado = int(x_a)
    y_acumulado = int(y_a)
    

    #matriz de translação
    translation_matrix = np.float32([[1,0,x_acumulado], [0,1,y_acumulado]])

    #Aplicar translação na imagem
    img_translation = cv2.warpAffine(frame2, translation_matrix, (num_cols, num_rows))
    
    #encontrar os novos valores de coluna e linhas 
    novo_cols = num_cols - abs(x_acumulado)
    novo_rows = num_rows - abs(y_acumulado)

    #encontrar a nova proporção
    novo_prop = novo_cols/novo_rows
    
    y = novo_rows
    x = novo_cols
    
    #se a nova proporção for menor que a antiga, corta a imagem em y
    if prop > novo_prop:
        #novo y equivalente a proporção
        y = int((novo_cols * num_rows) / num_cols)

    #caso a nova proporção for maior corta-se em x           
    else:
        #novo x equivalente a proporção
        x = int((novo_rows * num_cols )/ num_rows)
        
    
    #definir os valores de origem
    
    if y_acumulado<0:#se a imagem estiver subindo
        origem_y = num_rows - y + y_acumulado
    else: #se a imagem estiver descendo
        origem_y = num_rows - y
    
    if x_acumulado < 0:#se a imagem estiver indo para esquerda
        origem_x = num_cols - x + x_acumulado
    else: #se a imagem estiver indo para direita
        origem_x = num_cols - x

    #cortar a imagem com os determinados valores encontrados acima
    img_cut = img_translation[origem_x:x+origem_x, origem_y:y+origem_y]
    
    #encontrar os novos valors de coluna e linha
    cut_rows, cut_cols = img_cut.shape[:2]

    #matriz de translação
    translation_matrix2 = np.float32([[1,0,int(-origem_x/2)], [0,1,int(-origem_y/2)]])


    #Aplicar translação na imagem
    img_translation2 = cv2.warpAffine(img_cut, translation_matrix2, (cut_cols, cut_rows))
                
    #imagem cortada nas proporções desejada, mas com tamanho de janela não variável
    img_res = cv2.resize(img_translation2, (num_cols, num_rows), interpolation = cv2.INTER_CUBIC)
    
    #mostrar a imagem redimensionada e cortada
    cv2.imshow('Final', img_res)

    #condição para sair da janela da camera
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    #transformar a imagem anterior na atual imagem
    prvs_center = next1_center
    
#desligar captura de camera e fechar as janelas
captura.release()
cv2.destroyAllWindows()

In [ ]:
captura.release()